In [2]:
import cv2

img = cv2.imread("../input/yoga-poses-dataset/DATASET/TRAIN/goddess/00000097.jpg")

img.shape

In [3]:
import matplotlib.pyplot as plt

plt.imshow(img)

In [4]:
train_dir = "../input/yoga-poses-dataset/DATASET/TRAIN"
test_dir = "../input/yoga-poses-dataset/DATASET/TEST"

## **Image Processing**

In [5]:
import tensorflow as tf

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.,                                                                                                                   
                                                           rotation_range=0.2,
                                                           width_shift_range=0.2,
                                                           height_shift_range=0.2,
                                                           zoom_range = 0.2, 
                                                           horizontal_flip=True)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale= 1/255.)

In [6]:
train_data = train_gen.flow_from_directory(train_dir,
                                           target_size = (224,224),
                                           color_mode = "rgb",
                                           class_mode = "categorical",
                                           batch_size = 32
                                           )

test_data = test_gen.flow_from_directory(test_dir,
                                         target_size = (224,224),
                                         color_mode = "rgb",
                                         class_mode = "categorical",
                                         batch_size = 32
                                         )

In [7]:
labels = list(train_data.class_indices.keys())

labels

In [8]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [9]:
# visualizing the train data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):
  image, label = train_data.next()
  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(labels[tf.argmax(label[i])]) 
  plt.axis("off")

In [17]:
# building a CNN model

import tensorflow as tf
from tensorflow.keras import layers

model = tf.keras.Sequential([
                             layers.Conv2D(filters= 64, kernel_size= 2, activation= "relu", input_shape=(224,224,3)),
                             layers.MaxPooling2D(pool_size=2),

                             layers.Conv2D(filters= 64, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Conv2D(filters= 64, kernel_size= 2, activation= "relu"),
                             layers.MaxPooling2D(pool_size= 2),

                             layers.Flatten(),

                             layers.Dense(512, activation= "relu"),

                             layers.Dense(512, activation = "relu"),

                             layers.Dense(512, activation = "relu"),

                             layers.Dense(5, activation="softmax")
])

In [18]:
model.summary()

In [19]:
# compiling the model

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(),
    metrics = ["accuracy"]
)

In [20]:
# fitting data to the model
model.fit(train_data,
          epochs = 20,
          steps_per_epoch = len(train_data),
          validation_data = test_data,
          validation_steps = len(test_data)
          )

In [21]:
model_evaluation = model.evaluate(test_data)

In [22]:
print(f"Model Accuracy: {model_evaluation[1] * 100 : 0.2f} %")

In [23]:
# visualizing and predicting the  test data

import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(16,16))

for i in range(18):
  image,lable = test_data.next()
  model_pred = model.predict(image)
  plt.subplot(6,3,i+1)
  plt.imshow(image[i])
  plt.title(f"Prediction: {labels[tf.argmax(model_pred[i])]}\nOriginal: {labels[tf.argmax(label[i])]}")
  plt.subplots_adjust(top=1.25)
  plt.axis("off")